In [1]:
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]="0, 1, 2, 4"

In [2]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
from dask import dataframe as dd
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import log_loss
import optuna
import gc
xgb.__version__

'1.6.2'

In [3]:
!nvidia-smi

Tue Dec 13 07:12:18 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA TITAN RTX    On   | 00000000:09:00.0 Off |                  N/A |
| 41%   32C    P8    32W / 280W |    905MiB / 24220MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA TITAN RTX    On   | 00000000:41:00.0  On |                  N/A |
| 41%   

In [4]:
!nvidia-smi -L

GPU 0: NVIDIA TITAN RTX (UUID: GPU-0eb32f58-b8d5-17c0-e952-f4ec26a9353f)
GPU 1: NVIDIA TITAN RTX (UUID: GPU-50aeb092-88f5-4e0b-7f73-32741666f319)


In [5]:
train_x = dd.read_csv('../input/otto_ii/xgtrain_5.csv')
test_x = dd.read_csv('../input/otto_ii/xgval.csv')

In [6]:
train_y = train_x['target']
train_x = train_x[train_x.columns.difference(['target'])]

test_y = test_x['target']
test_x = test_x[test_x.columns.difference(['target'])]

In [7]:
cluster = LocalCUDACluster(n_workers=2)
client = Client(cluster)

2022-12-13 07:12:55,351 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2022-12-13 07:12:55,351 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2022-12-13 07:12:55,353 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2022-12-13 07:12:55,353 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize


In [8]:
dtrain = xgb.dask.DaskDMatrix(client, train_x, train_y)

In [9]:
dtest = xgb.dask.DaskDMatrix(client, test_x, test_y)


In [10]:
num_round = 1000

In [11]:
def objective(trial):
        
    params = {
        'objective': trial.suggest_categorical('objective',['multi:softprob']),
        'tree_method': trial.suggest_categorical('tree_method',['gpu_hist']),  # 'gpu_hist','hist'
        'lambda': trial.suggest_loguniform('lambda',1e-3,10.0),
        'alpha': trial.suggest_loguniform('alpha',1e-3,10.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.3,1.0),
        'subsample': trial.suggest_uniform('subsample', 0.4, 1.0),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001,0.1),
        #'n_estimators': trial.suggest_categorical('n_estimators', [1000]),
        'max_depth': trial.suggest_int('max_depth', 2, 25),
        #'random_state': trial.suggest_categorical('random_state', [24,48,2020]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1,300),
        'eval_metric': trial.suggest_categorical('eval_metric',['logloss']),
        'num_class':9,

    }

    output = xgb.dask.train(client, params, dtrain, num_round)
    
    booster = output['booster']  # booster is the trained model
    booster.set_param({'predictor': 'gpu_predictor'})

    predictions = xgb.dask.predict(client, booster, dtest)
    
    predictions = predictions.compute()

    lloss = log_loss(test_y, predictions)
    
    return lloss

In [12]:
study = optuna.create_study(direction='minimize')

[I 2022-12-13 07:13:03,248] A new study created in memory with name: no-name-1ec80095-a78e-4086-9e1c-93526c065141


In [13]:
%%time
study.optimize(objective, n_trials=2)

/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[07:13:06] task [xgboost.dask-1]:tcp://127.0.0.1:33991 got new rank 0
[I 2022-12-13 07:13:50,620] Trial 0 finished with value: 0.6884117553227117 and parameters: {'objective': 'multi:softprob', 'tree_method': 'gpu_hist', 'lambda': 0.1414574594523903, 'alpha': 0.24387011534291794, 'colsample_bytree': 0.5032028077004254, 'subsample': 0.8395323620235415, 'learning_rate': 0.0047373280848025485, 'max_depth': 7, 'min_child_weight': 222, 'eval_metric': 'logloss'}. Best is trial 0 with value: 0.6884117553227117.
/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[07:13:50] task [xgboost.dask-1]:tcp://127.0.0.1:33991 got new rank 0
[I 2022-12-13 07:14:07,921] Trial 1 finished with value: 0.6250554

CPU times: user 3.17 s, sys: 754 ms, total: 3.93 s
Wall time: 1min 2s


In [14]:
%%time
study.optimize(objective, n_trials=100)

/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[07:14:07] task [xgboost.dask-1]:tcp://127.0.0.1:33991 got new rank 0
[I 2022-12-13 07:15:08,579] Trial 2 finished with value: 0.5965574435497852 and parameters: {'objective': 'multi:softprob', 'tree_method': 'gpu_hist', 'lambda': 0.7627913417293688, 'alpha': 0.052183584471804656, 'colsample_bytree': 0.9943408974149217, 'subsample': 0.4818218363394942, 'learning_rate': 0.0056239335893615815, 'max_depth': 8, 'min_child_weight': 66, 'eval_metric': 'logloss'}. Best is trial 2 with value: 0.5965574435497852.
/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[07:15:08] task [xgboost.dask-1]:tcp://127.0.0.1:33991 got new rank 0
[I 2022-12-13 07:16:40,006] Trial 3 finished with value: 1.2308994

CPU times: user 16min 15s, sys: 5min 19s, total: 21min 35s
Wall time: 8h 37min 24s


In [15]:
study.best_trial.params

{'objective': 'multi:softprob',
 'tree_method': 'gpu_hist',
 'lambda': 0.007378765979072933,
 'alpha': 0.2637523408799014,
 'colsample_bytree': 0.6173101275749113,
 'subsample': 0.5603896325897884,
 'learning_rate': 0.025343425260582968,
 'max_depth': 19,
 'min_child_weight': 28,
 'eval_metric': 'logloss'}

In [16]:
real_test_x = dd.read_csv('../input/otto_ii/xgtest.csv')

In [17]:
real_test_y = real_test_x['target']
real_test_x = real_test_x[real_test_x.columns.difference(['target'])]

In [18]:
drealtest = xgb.dask.DaskDMatrix(client, real_test_x, real_test_y)

In [19]:
params = study.best_trial.params
params['num_class']= 9

output = xgb.dask.train(client, params, dtrain, num_round)
    
booster = output['booster']  # booster is the trained model
booster.set_param({'predictor': 'gpu_predictor'})


predictions = xgb.dask.predict(client, booster, drealtest)
    
predictions = predictions.compute()

log_loss(real_test_y, predictions)

/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[15:53:50] task [xgboost.dask-1]:tcp://127.0.0.1:33991 got new rank 0


0.4415311621477472

In [20]:
%%time
params = study.best_trial.params
params['seed'] = 2022
params['num_class']= 9

output = xgb.dask.train(client, params, dtrain, num_round)
    
booster = output['booster']  # booster is the trained model
booster.set_param({'predictor': 'gpu_predictor'})

predictions = xgb.dask.predict(client, booster, drealtest)
    
predictions = predictions.compute()

log_loss(real_test_y, predictions)

/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[15:56:48] task [xgboost.dask-1]:tcp://127.0.0.1:33991 got new rank 0


CPU times: user 5.78 s, sys: 2.38 s, total: 8.16 s
Wall time: 3min


0.44308164666584293

In [21]:
%%time 
scores = []

for jj in range(15):
    print(jj)
    params = study.best_trial.params
    params['num_class']= 9
    params['seed'] = 5*jj**3 + 137

    output = xgb.dask.train(client, params, dtrain, num_round)

    booster = output['booster']  # booster is the trained model
    booster.set_param({'predictor': 'gpu_predictor'})

    predictions = xgb.dask.predict(client, booster, drealtest)

    predictions = predictions.compute()
    
    error = log_loss(real_test_y, predictions)

    scores.append(error)
    print(error)

0


/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[15:59:48] task [xgboost.dask-1]:tcp://127.0.0.1:33991 got new rank 0


0.44127668090410527
1


/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[16:02:47] task [xgboost.dask-1]:tcp://127.0.0.1:33991 got new rank 0


0.44172401428855823
2


...

0.4420653176842819
13


/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[16:38:32] task [xgboost.dask-1]:tcp://127.0.0.1:33991 got new rank 0


0.4426646724383337
14


/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[16:41:29] task [xgboost.dask-1]:tcp://127.0.0.1:33991 got new rank 0


0.4435280541834005
CPU times: user 1min 31s, sys: 28.7 s, total: 2min
Wall time: 44min 39s


In [22]:
scores

[0.44127668090410527,
 0.44172401428855823,
 0.44315257490927185,
 0.441560446464799,
 0.44231034972320993,
 0.4426254269772641,
 0.4418859970187513,
 0.44205585185847884,
 0.4424926259210848,
 0.44287242564555246,
 0.44224248864884297,
 0.4437120623053991,
 0.4420653176842819,
 0.4426646724383337,
 0.4435280541834005]

In [23]:
import numpy as np
np.mean(scores)

0.44241126593142227

2022-12-13 18:04:06,746 - distributed.nanny - ERROR - Worker process died unexpectedly
/home/tunguz/.local/lib/python3.8/site-packages/distributed/worker_state_machine.py:3649: FutureWarning: The `Worker.nthreads` attribute has been moved to `Worker.state.nthreads`
  warnings.warn(
/home/tunguz/.local/lib/python3.8/site-packages/distributed/worker_state_machine.py:3649: FutureWarning: The `Worker.nthreads` attribute has been moved to `Worker.state.nthreads`
  warnings.warn(
/home/tunguz/.local/lib/python3.8/site-packages/distributed/worker_state_machine.py:3649: FutureWarning: The `Worker.nthreads` attribute has been moved to `Worker.state.nthreads`
  warnings.warn(
/home/tunguz/.local/lib/python3.8/site-packages/distributed/worker_state_machine.py:3649: FutureWarning: The `Worker.nthreads` attribute has been moved to `Worker.state.nthreads`
  warnings.warn(
/home/tunguz/.local/lib/python3.8/site-packages/distributed/worker_state_machine.py:3649: FutureWarning: The `Worker.nthreads` at